In [1]:
# Author: Sunny You
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras

2023-09-07 18:39:47.928796: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.chdir('/Users/sunnyyou/Documents/Real_Time_HAI/HPCNN/IMERG')

In [3]:
train = pd.read_csv("IC_Data/ATL/P24_train_resample.csv")
train = train[["GIS_ID", "DATE", "SHIPS_PER", "SHIPS_POT", "VMAX", "SHDC", "IC", "Category"]]
# train = train.drop(["VMAX_FT"], axis = 1)
train.columns = ["GIS_ID", "DATE", "PER", "POT", "VMAX", "SHDC_FT", "IC", "Category"]
train

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category
0,ATL_200002_TD_2000062318,2000062318,9.310000e-09,84.0,30,10.100000,0,TD
1,ATL_200002_TD_2000062400,2000062400,0.000000e+00,84.0,30,4.300000,0,TD
2,ATL_200002_TD_2000062406,2000062406,0.000000e+00,84.0,30,4.400000,0,TD
3,ATL_200002_TD_2000062412,2000062412,0.000000e+00,85.0,30,5.700000,0,TD
4,ATL_200002_TD_2000062418,2000062418,0.000000e+00,87.0,30,6.200000,-5,TD
...,...,...,...,...,...,...,...,...
7411,ATL_201913_C4_2019092712,2019092712,-5.000000e+00,6.0,120,18.800001,-15,Maj
7412,ATL_201905_C3_2019090312,2019090312,-1.500000e+01,33.0,100,8.400001,-10,Maj
7413,ATL_201408_C4_2014101500,2014101500,2.000000e+01,39.0,115,6.700000,-5,Maj
7414,ATL_201511_C4_2015100212,2015100212,-5.000000e+00,40.0,115,25.500000,20,Maj


In [4]:
# test = pd.read_csv("IC_Data/P18_2018_test.csv")
# test = test[["GIS_ID", "DATE", "SHIPS_PER", "SHIPS_POT", "VMAX", "SHDC", "IC", "Category"]]
# test = test.drop(["VMAX_FT"], axis = 1)
# test.columns = ["GIS_ID", "DATE", "PER", "POT", "VMAX", "SHDC_FT", "IC", "Category"]
# test

In [5]:
# train = pd.concat([train, test])
test = pd.DataFrame(np.reshape(['ATL_202313_C3_2023090718', '2023090718', 35, 54, 105, 9.9, -999, 'C3'], (1, 8)))
test.columns = ('GIS_ID', 'DATE', 'PER', 'POT', 'VMAX', 'SHDC_FT', 'IC', 'Category')
train = train.iloc[:, :-1]
train = train.reset_index(drop = False)
print(train.head(), train.shape)
print(test.head(), test.shape)

   index                    GIS_ID        DATE           PER   POT  VMAX   
0      0  ATL_200002_TD_2000062318  2000062318  9.310000e-09  84.0    30  \
1      1  ATL_200002_TD_2000062400  2000062400  0.000000e+00  84.0    30   
2      2  ATL_200002_TD_2000062406  2000062406  0.000000e+00  84.0    30   
3      3  ATL_200002_TD_2000062412  2000062412  0.000000e+00  85.0    30   
4      4  ATL_200002_TD_2000062418  2000062418  0.000000e+00  87.0    30   

   SHDC_FT  IC  
0     10.1   0  
1      4.3   0  
2      4.4   0  
3      5.7   0  
4      6.2  -5   (7416, 8)
                     GIS_ID        DATE PER POT VMAX SHDC_FT    IC Category
0  ATL_202313_C3_2023090718  2023090718  35  54  105     9.9  -999       C3 (1, 8)


In [6]:
train_img = []
train_ships = []
train_label = []
test_img = []
test_ships = []
test_label = []
for f in range(len(train.GIS_ID)) :
    filename = "IMERG_CSV/" + train.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.IC[f]
        train_label.append(lab)
        ships = np.array([train.VMAX[f], train.POT[f], train.PER[f], train.SHDC_FT[f]])
        train_ships.append(ships)
    except Exception as e:
        pass

for f in range(len(test.GIS_ID)) :
    filename = "IMERG_CSV/" + test.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.IC[f]
        test_label.append(lab)
        ships = np.array([test.VMAX[f], test.POT[f], test.PER[f], test.SHDC_FT[f]])
        test_ships.append(ships)
    except Exception as e:
        pass

In [7]:
print(len(train_img))
print(len(train_ships))
print(len(train_label))
print(len(test_img))
print(len(test_ships))
print(len(test_label))

7416
7416
7416
1
1
1


In [8]:
test_ships = np.float64(test_ships)
test_label = np.float64(test_label)

In [9]:
X_train_img = train_img
X_train_ships = train_ships
y_train = train_label
X_test_img = test_img
X_test_ships = test_ships
y_test = test_label

In [10]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,41,41,1)
X_train_img = X_train_img.astype('float32')
X_train_ships = np.array(X_train_ships)
X_train_ships = X_train_ships.reshape(-1,4)
y_train = np.array(y_train)

In [11]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,41,41,1)
X_test_img = X_test_img.astype('float32')
X_test_ships = np.array(X_test_ships)
X_test_ships = X_test_ships.reshape(-1,4)
y_test = np.array(y_test)

In [12]:
ships_input = keras.Input(shape =(4,), name = "ships_layer")
img_input = keras.Input(shape =(41, 41, 1), name = "img_layer")

w = keras.layers.Conv2D(64,8) (img_input)
w = keras.layers.Conv2D(64,8)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, ships_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 8) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,5)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,3)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, ships_input])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (6,2))(img_input)
y = keras.layers.Conv2D(256, (2,6))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,3)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, ships_input])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 7)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, ships_input])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_layer (InputLayer)         [(None, 41, 41, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 34, 34, 64)   4160        ['img_layer[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)              (None, 27, 27, 64)   262208      ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 27, 27, 64)   4160        ['conv2d_1[0][0]']               
                                                                                            

In [13]:
new_model1.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model3.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model4.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

In [14]:
new_model3.fit([X_train_img, X_train_ships], y_train, epochs=3, batch_size=8, validation_split = 0.1)
res = new_model3.evaluate([X_test_img, X_test_ships], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))

Epoch 1/3
835/835 [==============================] - 217s 258ms/step - loss: 11.2563 - mae: 11.2563 - mse: 229.0041 - val_loss: 13.5974 - val_mae: 13.5974 - val_mse: 306.9965
Epoch 2/3
835/835 [==============================] - 231s 276ms/step - loss: 10.3689 - mae: 10.3689 - mse: 197.1802 - val_loss: 12.6571 - val_mae: 12.6571 - val_mse: 275.8770
Epoch 3/3
1/1 [==============================] - 0s 288ms/step - loss: 1014.3146 - mae: 1014.3146 - mse: 1028834.6250
MAE = 1014.3145751953125
RMSE = 1014.3148549636843


In [18]:
preds = new_model3.predict([X_test_img, X_test_ships])
a = np.average(preds, axis = 1)
test['preds'] = a

1/1 [==============================] - 0s 27ms/step


In [19]:
test

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category,preds
0,ATL_202313_C3_2023090718,2023090718,35,54,105,9.9,-999,C3,15.314613
